In [ ]:
import math

%load_ext Cython

from Bio import PDB

In [ ]:
#XXX
repository = PDB.PDBList()
parser = PDB.PDBParser()
repository.retrieve_pdb_file('1TUP', pdir='.', file_format='pdb')
p53_1tup = parser.get_structure('P 53', 'pdb1tup.ent')

In [ ]:
def get_distance(atoms):
    atoms = list(atoms)  # not great
    natoms = len(atoms)
    for i in range(natoms - 1):
        xi, yi, zi = atoms[i].coord
        for j in range(i + 1, natoms):
            xj, yj, zj = atoms[j].coord
            my_dist = math.sqrt((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2) 

In [ ]:
%time get_distance(p53_1tup.get_atoms())
#XXX time, not timeit

In [ ]:
%%cython
import math
def get_distance_cython_0(atoms):
    atoms = list(atoms)
    natoms = len(atoms)
    for i in range(natoms - 1):
        xi, yi, zi = atoms[i].coord
        for j in range(i + 1, natoms):
            xj, yj, zj = atoms[j].coord
            my_dist = math.sqrt((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2) 

In [ ]:
%time get_distance_cython_0(p53_1tup.get_atoms())

In [ ]:
%%cython
cimport cython
from libc.math cimport sqrt, pow

cdef double get_dist_cython(double xi, double yi, double zi,
                     double xj, double yj, double zj):
    return sqrt(pow(xi - xj, 2) + pow(yi - yj, 2) + pow(zi - zj, 2))

def get_distance_cython_1(object atoms):
    natoms = len(atoms)
    cdef double x1, xj, yi, yj, zi, zj
    for i in range(natoms - 1):
        xi, yi, zi = atoms[i]
        for j in range(i + 1, natoms):
            xj, yj, zj = atoms[j]
            my_dist = get_dist_cython(xi, yi, zi, xj, yj, zj)

In [ ]:
%timeit get_distance_cython_1([atom.coord for atom in p53_1tup.get_atoms()])

## Numba

In [ ]:
from numba import float_
from numba.decorators import jit

In [ ]:
get_distance_numba_0 = jit(get_distance)

In [ ]:
%time get_distance_numba_0(p53_1tup.get_atoms())

In [ ]:
@jit
def get_dist_numba(xi, yi, zi, xj, yj, zj):
    return math.sqrt((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2) 

def get_distance_numba_1(atoms):
    natoms = len(atoms)
    for i in range(natoms - 1):
        xi, yi, zi = atoms[i]
        for j in range(i + 1, natoms):
            xj, yj, zj = atoms[j]
            my_dist = get_dist_numba(xi, yi, zi, xj, yj, zj)

In [ ]:
%timeit get_distance_numba_1([atom.coord for atom in p53_1tup.get_atoms()])